In [1]:
import pandas as pd

df = pd.read_excel("Final_Merged_Venue_Date_Value_updated.xlsx")
print(df.columns)


Index(['Event code', 'Artist', 'Ticket price', 'Date', 'Tickets sold',
       'Solo or multiple', 'Venue', 'Capacity', 'Day', 'Days', 'Weekend',
       'coordinates', 'latitude', 'longitude', 'temperature',
       'feels_like_temp', 'temp_min', 'temp_max', 'precipitation_mm',
       'snow_mm', 'wind_speed', 'humidity', 'pressure', 'visibility',
       'cloudiness', 'weather_conditions', 'weather_description',
       'bad_weather_flag', 'temp_category', 'weather_risk_score',
       'Artist_matched', 'Artist.1', 'artist_id', 'cm_artist_score',
       'cm_artist_rank', 'country', 'record_label', 'primary_genre',
       'spotify_followers', 'spotify_monthly_listeners',
       'youtube_total_subscribers', 'youtube_top_country',
       'tiktok_total_followers', 'tiktok_top_country',
       'instagram_total_followers', 'instagram_top_country',
       'instagram_engagement_rate', 'albums_count', 'tracks_count',
       'events_count', 'riaa_awards_count', 'avg_ticket_price',
       'min_ticket_

In [2]:
print(df['Date'].dtype)
print(df['Date'].head())


datetime64[ns]
0   2021-10-29
1   2022-07-16
2   2022-03-19
3   2022-03-19
4   2022-07-16
Name: Date, dtype: datetime64[ns]


In [4]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

result = (
    df.groupby(['Venue', 'Year', 'Month'])['Tickets sold']
      .mean()
      .reset_index()
)

print(result)


                Venue  Year  Month  Tickets sold
0           9:30 Club  2021      9    768.214286
1           9:30 Club  2021     10   1087.222222
2           9:30 Club  2021     11   1104.916667
3           9:30 Club  2021     12   1218.800000
4           9:30 Club  2022      1   1056.000000
..                ...   ...    ...           ...
204  The Spot on Kirk  2025      5     46.200000
205  The Spot on Kirk  2025      6     46.625000
206  The Spot on Kirk  2025      7     43.428571
207  The Spot on Kirk  2025      8     53.500000
208  The Spot on Kirk  2025      9     38.000000

[209 rows x 4 columns]


In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(result)


                  Venue  Year  Month  Tickets sold
0             9:30 Club  2021      9    768.214286
1             9:30 Club  2021     10   1087.222222
2             9:30 Club  2021     11   1104.916667
3             9:30 Club  2021     12   1218.800000
4             9:30 Club  2022      1   1056.000000
5             9:30 Club  2022      2   1211.937500
6             9:30 Club  2022      3   1035.750000
7             9:30 Club  2022      4   1003.185185
8             9:30 Club  2022      5    990.352941
9             9:30 Club  2022      6    997.411765
10            9:30 Club  2022      7    747.615385
11            9:30 Club  2022      8    795.250000
12            9:30 Club  2022      9    998.263158
13            9:30 Club  2022     10   1001.103448
14            9:30 Club  2022     11   1028.772727
15            9:30 Club  2022     12   1081.692308
16            9:30 Club  2023      1    919.071429
17            9:30 Club  2023      2   1180.647059
18            9:30 Club  2023  

In [7]:
result['Tickets sold'] = result['Tickets sold'].round().astype(int)

result.to_json("venue_month_year_avg_tickets.json", orient="records")


In [8]:
result['Tickets sold'] = result['Tickets sold'].round().astype(int)

result.to_json(
    "venue_month_year_avg_tickets.json",
    orient="records",
    indent=4
)


In [10]:
last_dates = (
    df.groupby('Venue')['Date']
      .max()
      .reset_index()
)

last_dates['Year'] = last_dates['Date'].dt.year
last_dates['Month'] = last_dates['Date'].dt.month

print(last_dates)


                Venue       Date  Year  Month
0           9:30 Club 2025-09-28  2025      9
1  Bright Box Theater 2025-08-16  2025      8
2    Carolina Theatre 2025-10-01  2025     10
3    Millwald Theatre 2025-09-21  2025      9
4    The Spot on Kirk 2025-09-27  2025      9


In [11]:
import pandas as pd

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# find last date for each venue
last_dates = df.groupby('Venue')['Date'].max().reset_index()

rows = []

for _, row in last_dates.iterrows():
    venue = row['Venue']
    last_date = row['Date']

    temp = df[df['Venue'] == venue]

    periods = {
        'last_1_month': last_date - pd.DateOffset(months=1),
        'last_2_months': last_date - pd.DateOffset(months=2),
        'last_3_months': last_date - pd.DateOffset(months=3),
        'last_6_months': last_date - pd.DateOffset(months=6),
        'last_1_year': last_date - pd.DateOffset(years=1)
    }

    out = {'Venue': venue}

    for label, start_date in periods.items():
        mask = (temp['Date'] > start_date) & (temp['Date'] <= last_date)
        out[label] = temp.loc[mask, 'Tickets sold'].mean()

    rows.append(out)

result = pd.DataFrame(rows)
result['last_1_month'] = result['last_1_month'].round()
result['last_2_months'] = result['last_2_months'].round()
result['last_3_months'] = result['last_3_months'].round()
result['last_6_months'] = result['last_6_months'].round()
result['last_1_year'] = result['last_1_year'].round()

print(result)


                Venue  last_1_month  last_2_months  last_3_months  \
0           9:30 Club         966.0          877.0          860.0   
1  Bright Box Theater          60.0           54.0           62.0   
2    Carolina Theatre         520.0          609.0          622.0   
3    Millwald Theatre           0.0            0.0            0.0   
4    The Spot on Kirk          38.0           43.0           43.0   

   last_6_months  last_1_year  
0          880.0        905.0  
1           77.0         85.0  
2          688.0        708.0  
3           95.0        133.0  
4           44.0         47.0  


In [12]:
result

,Venue,last_1_month,last_2_months,last_3_months,last_6_months,last_1_year
0,9:30 Club,966.0,877.0,860.0,880.0,905.0
1,Bright Box Theater,60.0,54.0,62.0,77.0,85.0
2,Carolina Theatre,520.0,609.0,622.0,688.0,708.0
3,Millwald Theatre,0.0,0.0,0.0,95.0,133.0
4,The Spot on Kirk,38.0,43.0,43.0,44.0,47.0


In [18]:
import pandas as pd

# ensure date column is datetime
df['Date'] = pd.to_datetime(df['Date'])

# ensure tickets sold is numeric
df['Tickets sold'] = pd.to_numeric(df['Tickets sold'], errors='coerce').fillna(0).astype(int)

# last date per venue
last_dates = df.groupby('Venue')['Date'].max().reset_index()

# get venue capacity (assuming constant per venue)
venue_capacity = df.groupby('Venue')['Capacity'].first().reset_index()

rows = []

for _, row in last_dates.iterrows():
    venue = row['Venue']
    last_date = row['Date']

    temp = df[df['Venue'] == venue].copy()

    periods = {
        'last_1_month': last_date - pd.DateOffset(months=1),
        'last_2_months': last_date - pd.DateOffset(months=2),
        'last_3_months': last_date - pd.DateOffset(months=3),
        'last_6_months': last_date - pd.DateOffset(months=6),
        'last_1_year': last_date - pd.DateOffset(years=1)
    }

    out = {'Venue': venue, 'last_date': last_date}

    for label, start_date in periods.items():
        # inclusive on both ends: >= start_date and <= last_date
        mask = (temp['Date'] >= start_date) & (temp['Date'] <= last_date)
        out[label] = int(temp.loc[mask, 'Tickets sold'].sum())  # sum and cast to int

        # also store count of events in that window for debugging if needed
        out[f'{label}_event_count'] = int(mask.sum())

    rows.append(out)

result = pd.DataFrame(rows)

# merge capacity into final table
result = result.merge(venue_capacity, on='Venue', how='left')

# reorder columns: Venue, Capacity, last_date, metrics
cols = ['last_1_month', 'last_2_months', 'last_3_months', 'last_6_months', 'last_1_year']
event_counts = [c + '_event_count' for c in cols]
result = result[['Venue', 'Capacity', 'last_date'] + cols + event_counts]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

result


,Venue,Capacity,last_date,last_1_month,last_2_months,last_3_months,last_6_months,last_1_year,last_1_month_event_count,last_2_months_event_count,last_3_months_event_count,last_6_months_event_count,last_1_year_event_count
0,9:30 Club,1200,2025-09-28,16422,26305,33533,108641,218785,17,30,39,124,241
1,Bright Box Theater,300,2025-08-16,417,868,1705,3992,8328,7,16,27,52,98
2,Carolina Theatre,1055,2025-10-01,3118,5481,6217,12390,29721,6,9,10,18,42
3,Millwald Theatre,500,2025-09-21,0,0,0,2001,4463,4,7,8,21,34
4,The Spot on Kirk,125,2025-09-27,304,518,872,2133,4062,8,12,20,48,87


In [19]:
# ---- Debug: inspect 9:30 Club (or any venue) rows contributing to each window ----
venue_to_check = 'Millwald Theatre'   # change this to any venue you want to inspect
if venue_to_check in last_dates['Venue'].values:
    last_date = last_dates.loc[last_dates['Venue'] == venue_to_check, 'Date'].iloc[0]
    temp = df[df['Venue'] == venue_to_check].sort_values('Date')
    print(f"\nDebug rows for {venue_to_check} (last_date = {last_date.date()}):\n")
    print(temp[['Date', 'Event code', 'Tickets sold']])

    for label, offset_months in [('last_1_month', 1), ('last_2_months', 2), ('last_3_months', 3), ('last_6_months', 6), ('last_1_year', 12)]:
        start = last_date - pd.DateOffset(months=offset_months) if label != 'last_1_year' else last_date - pd.DateOffset(years=1)
        mask = (temp['Date'] >= start) & (temp['Date'] <= last_date)
        s = temp.loc[mask, 'Tickets sold'].sum()
        cnt = mask.sum()
        print(f"{label}: start={start.date()}, end={last_date.date()}, events={cnt}, tickets_sum={s}")
else:
    print(f"\nVenue {venue_to_check} not found in data.")



Debug rows for Millwald Theatre (last_date = 2025-09-21):

           Date  Event code  Tickets sold
1804 2024-01-21       20002           431
1805 2024-01-25       20003            95
1806 2024-01-26       20004           282
1807 2024-02-03       20005           111
1808 2024-02-17       20009           219
1809 2024-03-01       20010            93
1810 2024-03-29       20018           203
1811 2024-04-12       20019           115
1812 2024-05-03       20027            95
1813 2024-05-10       20028           166
1814 2024-05-17       20029           278
1815 2024-08-16       20048             0
1816 2024-08-17       20049             0
1817 2024-08-24       20050             0
1818 2024-09-07       20053           113
1819 2024-09-21       20058            79
1820 2024-10-05       20061            22
1821 2024-11-02       20065           465
1822 2024-11-16       20067           260
1823 2024-12-04       20069           221
1824 2025-01-17       20073           280
1825 2025-01-18 

In [21]:
import pandas as pd

# Ensure types
df['Date'] = pd.to_datetime(df['Date'])
df['Tickets sold'] = pd.to_numeric(df['Tickets sold'], errors='coerce').fillna(0).astype(int)

venues = df['Venue'].unique()

rows = []

for venue in venues:
    temp = df[df['Venue'] == venue].copy()

    # Most recent event at this venue
    last_date = temp['Date'].max()

    # Windows
    start_1m = last_date - pd.DateOffset(months=1)
    start_1y = last_date - pd.DateOffset(years=1)

    # ----------------------------
    # LAST 1 MONTH
    # ----------------------------
    mask_1m = (temp['Date'] >= start_1m) & (temp['Date'] <= last_date)

    total_1m = temp.loc[mask_1m, 'Tickets sold'].sum()
    events_1m = mask_1m.sum()
    avg_1m = round(total_1m / events_1m, 2) if events_1m > 0 else None

    # ----------------------------
    # LAST 1 YEAR
    # ----------------------------
    mask_1y = (temp['Date'] >= start_1y) & (temp['Date'] <= last_date)

    total_1y = temp.loc[mask_1y, 'Tickets sold'].sum()
    events_1y = mask_1y.sum()
    avg_1y = round(total_1y / events_1y, 2) if events_1y > 0 else None

    # ----------------------------
    # STORE RESULT
    # ----------------------------
    rows.append({
        "venue": venue,
        "last_play_date": str(last_date.date()),

        "tickets_last_1_month": int(total_1m),
        "events_last_1_month": int(events_1m),
        "avg_tickets_last_1_month": avg_1m,

        "tickets_last_1_year": int(total_1y),
        "events_last_1_year": int(events_1y),
        "avg_tickets_last_1_year": avg_1y
    })

# Convert to table
venue_summary = pd.DataFrame(rows)

# Show full table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

venue_summary


,venue,last_play_date,tickets_last_1_month,events_last_1_month,avg_tickets_last_1_month,tickets_last_1_year,events_last_1_year,avg_tickets_last_1_year
0,The Spot on Kirk,2025-09-27,304,8,38.00,4062,87,46.69
1,Bright Box Theater,2025-08-16,417,7,59.57,8328,98,84.98
2,Carolina Theatre,2025-10-01,3118,6,519.67,29721,42,707.64
3,9:30 Club,2025-09-28,16422,17,966.00,218785,241,907.82
4,Millwald Theatre,2025-09-21,0,4,0.00,4463,34,131.26


In [23]:
import pandas as pd

# Ensure types
df['Date'] = pd.to_datetime(df['Date'])
df['Tickets sold'] = pd.to_numeric(df['Tickets sold'], errors='coerce').fillna(0).astype(int)

# Get capacity per venue (assumed constant)
venue_capacity = df.groupby('Venue')['Capacity'].first()

rows = []

venues = df['Venue'].unique()

for venue in venues:
    temp = df[df['Venue'] == venue].copy()

    # Most recent event date for this venue
    last_date = temp['Date'].max()

    # ------- Windows --------
    start_1m = last_date - pd.DateOffset(months=1)
    start_1y = last_date - pd.DateOffset(years=1)
    start_6m = last_date - pd.DateOffset(months=6)

    # ----------------------------
    # LAST 1 MONTH SUMMARY
    # ----------------------------
    mask_1m = (temp['Date'] >= start_1m) & (temp['Date'] <= last_date)

    total_1m = temp.loc[mask_1m, 'Tickets sold'].sum()
    events_1m = mask_1m.sum()
    avg_1m = round(total_1m / events_1m, 2) if events_1m > 0 else None

    # ----------------------------
    # LAST 1 YEAR SUMMARY
    # ----------------------------
    mask_1y = (temp['Date'] >= start_1y) & (temp['Date'] <= last_date)

    total_1y = temp.loc[mask_1y, 'Tickets sold'].sum()
    events_1y = mask_1y.sum()
    avg_1y = round(total_1y / events_1y, 2) if events_1y > 0 else None

    # ----------------------------
    # LAST 6-MONTH MONTH-BY-MONTH HISTORY
    # ----------------------------
    temp['Year'] = temp['Date'].dt.year
    temp['Month'] = temp['Date'].dt.month
    temp['Month_Name'] = temp['Date'].dt.strftime('%b')

    mask_6m = (temp['Date'] >= start_6m) & (temp['Date'] <= last_date)

    hist = (
        temp.loc[mask_6m]
        .groupby(['Year', 'Month', 'Month_Name'])['Tickets sold']
        .sum()
        .reset_index()
        .sort_values(['Year', 'Month'], ascending=[True, True])
    )

    # Convert to JSON-ready list
    history_6m = [
        {
            "month": row['Month_Name'],
            "tickets": int(row['Tickets sold'])
        }
        for _, row in hist.iterrows()
    ]

    # ----------------------------
    # FINAL ROW OUTPUT
    # ----------------------------
    rows.append({
        "venue": venue,
        "capacity": int(venue_capacity[venue]),
        "last_play_date": str(last_date.date()),

        "tickets_last_1_month": int(total_1m),
        "events_last_1_month": int(events_1m),
        "avg_tickets_last_1_month": avg_1m,

        "tickets_last_1_year": int(total_1y),
        "events_last_1_year": int(events_1y),
        "avg_tickets_last_1_year": avg_1y,

        "history_last_6_months": history_6m
    })

# Convert to DataFrame for table display
venue_summary = pd.DataFrame(rows)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

venue_summary


,venue,capacity,last_play_date,tickets_last_1_month,events_last_1_month,avg_tickets_last_1_month,tickets_last_1_year,events_last_1_year,avg_tickets_last_1_year,history_last_6_months
0,The Spot on Kirk,125,2025-09-27,304,8,38.00,4062,87,46.69,"[{'month': 'Mar', 'tickets': 106}, {'month': '..."
1,Bright Box Theater,300,2025-08-16,417,7,59.57,8328,98,84.98,"[{'month': 'Feb', 'tickets': 524}, {'month': '..."
2,Carolina Theatre,1055,2025-10-01,3118,6,519.67,29721,42,707.64,"[{'month': 'Apr', 'tickets': 2869}, {'month': ..."
3,9:30 Club,1200,2025-09-28,16422,17,966.00,218785,241,907.82,"[{'month': 'Mar', 'tickets': 3141}, {'month': ..."
4,Millwald Theatre,500,2025-09-21,0,4,0.00,4463,34,131.26,"[{'month': 'Mar', 'tickets': 270}, {'month': '..."


In [24]:
import json

# Convert DataFrame rows to a clean Python list
data = venue_summary.to_dict(orient="records")

# Save as pretty JSON
with open("venue_summary.json", "w") as f:
    json.dump(data, f, indent=4)


In [1]:
import pandas as pd

df = pd.read_excel("Final_Merged_Venue_Date_Value_updated.xlsx")
print(df.columns)


Index(['Event code', 'Artist', 'Ticket price', 'Date', 'Tickets sold',
       'Solo or multiple', 'Venue', 'Capacity', 'Day', 'Days', 'Weekend',
       'coordinates', 'latitude', 'longitude', 'temperature',
       'feels_like_temp', 'temp_min', 'temp_max', 'precipitation_mm',
       'snow_mm', 'wind_speed', 'humidity', 'pressure', 'visibility',
       'cloudiness', 'weather_conditions', 'weather_description',
       'bad_weather_flag', 'temp_category', 'weather_risk_score',
       'Artist_matched', 'Artist.1', 'artist_id', 'cm_artist_score',
       'cm_artist_rank', 'country', 'record_label', 'primary_genre',
       'spotify_followers', 'spotify_monthly_listeners',
       'youtube_total_subscribers', 'youtube_top_country',
       'tiktok_total_followers', 'tiktok_top_country',
       'instagram_total_followers', 'instagram_top_country',
       'instagram_engagement_rate', 'albums_count', 'tracks_count',
       'events_count', 'riaa_awards_count', 'avg_ticket_price',
       'min_ticket_

In [2]:
df.head()

,Event code,Artist,Ticket price,Date,Tickets sold,Solo or multiple,Venue,Capacity,Day,Days,...,instagram_engagement_rate,albums_count,tracks_count,events_count,riaa_awards_count,avg_ticket_price,min_ticket_price,max_ticket_price,Competitive_Value,event
0,30138,Harvest Blaque,5.0,2021-10-29,24,1,The Spot on Kirk,125,5,Fri,...,NaN,14.0,10.0,0.0,0.0,NaN,NaN,NaN,No data,2.0
1,30184,Harvest Blaque,10.0,2022-07-16,30,2,The Spot on Kirk,125,6,Sat,...,NaN,14.0,10.0,0.0,0.0,NaN,NaN,NaN,No data,1.0
2,10035,Derrick Knopsnyder,20.0,2022-03-19,83,1,Bright Box Theater,300,6,Sat,...,NaN,1.0,10.0,0.0,0.0,NaN,NaN,NaN,No data,2.0
3,10037,Derrick Knopsnyder,18.0,2022-03-19,64,1,Bright Box Theater,300,6,Sat,...,NaN,1.0,10.0,0.0,0.0,NaN,NaN,NaN,No data,2.0
4,10093,Kevin Lee,20.0,2022-07-16,90,1,Bright Box Theater,300,6,Sat,...,NaN,1.0,10.0,0.0,0.0,NaN,NaN,NaN,No data,1.0


In [4]:
import pandas as pd

# Make sure Date is datetime
df['Date'] = pd.to_datetime(df['Date'])

# Build Artist–Venue stats
artist_venue_stats = (
    df.groupby(['Artist', 'Venue'])
      .agg(
          last_play_date=('Date', 'max'),
          times_played=('Date', 'count')
      )
      .reset_index()
)

# Convert datetime to string for JSON saving
artist_venue_stats['last_play_date'] = artist_venue_stats['last_play_date'].dt.strftime('%Y-%m-%d')

# Save JSON in the SAME folder as your notebook
artist_venue_stats.to_json(
    "artist_venue_last_play.json",
    orient="records",
    indent=4
)

print("Saved artist_venue_last_play.json")


Saved artist_venue_last_play.json


In [5]:
import pandas as pd
import json

# df = your cleaned dataframe that contains Artist + social media columns

# all social + chartmetric fields to include
social_cols = [
    'artist_id',
    'cm_artist_score',
    'cm_artist_rank',
    'spotify_followers',
    'spotify_monthly_listeners',
    'youtube_total_subscribers',
    'youtube_top_country',
    'tiktok_total_followers',
    'tiktok_top_country',
    'instagram_total_followers',
    'instagram_top_country',
    'instagram_engagement_rate',
    'country',
    'record_label',
    'primary_genre',
    'albums_count',
    'tracks_count',
    'events_count',
    'riaa_awards_count'
]

# Only include columns that actually exist in your dataframe
existing_cols = [c for c in social_cols if c in df.columns]

artist_json = {}

# GROUP BY ARTIST
for artist, group in df.groupby("Artist"):
    
    # take the first row (or adjust logic if needed)
    row = group[existing_cols].iloc[0].to_dict()
    
    # add artist name inside the object
    row["artist"] = artist
    
    # clean numpy types and NaN
    clean_row = {k: (None if pd.isna(v) else v) for k, v in row.items()}
    
    # store using artist name as key
    artist_json[artist] = clean_row

# SAVE JSON FILE
with open("artist_social.json", "w") as f:
    json.dump(artist_json, f, indent=4)

print("Saved artist_social.json with", len(artist_json), "artists.")


Saved artist_social.json with 1336 artists.
